In [23]:


import pandas as pd
import numpy as np
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras


from func_data import*

In [24]:
df=pd.read_csv(r"D:\data\diabetes_012_health_indicators_BRFSS2015.csv")
df.rename(columns={"Diabetes_012": "Diabetes"}, inplace=True)

map = {
    0: 0,
    1: 1,
    2: 1,
    
}
df["Diabetes"] = df["Diabetes"].replace(map)
df.dropna
df=df.sample(50000)

In [25]:
neg, pos = np.bincount(df['Diabetes'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 50000
    Positive: 7918 (15.84% of total)



In [26]:
Y=df['Diabetes'].values
X=df.drop(['Diabetes'],axis=1)

In [27]:
data_split=dataset_split(X,Y,test_size=0.15)
X_train,X_test,Y_train,Y_test=data_split

In [28]:
from tensorboard.plugins.hparams import api as hp

In [29]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16,32,64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.5, 0.8))


METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )



In [30]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    verbose=1,
    patience=20,
    mode='max',
    restore_best_weights=True)

In [31]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
                        
  ])
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=['accuracy'],
  )



  model.fit(X_train, Y_train, epochs=25) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(X_test, Y_test)
  return accuracy


In [32]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [33]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    
    hparams = {
        HP_NUM_UNITS: num_units,
        HP_DROPOUT: dropout_rate,
        }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning3/' + run_name, hparams)
    session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.5}
Epoch 1/25


1329/1329 [==============================] - 2s 1ms/step - loss: 0.6396 - accuracy: 0.7811
Epoch 2/25
1329/1329 [==============================] - 2s 1ms/step - loss: 0.4072 - accuracy: 0.8415
Epoch 3/25
1329/1329 [==============================] - 2s 1ms/step - loss: 0.3864 - accuracy: 0.8422
Epoch 4/25
1329/1329 [==============================] - 1s 936us/step - loss: 0.3769 - accuracy: 0.8425
Epoch 5/25
1329/1329 [==============================] - 1s 805us/step - loss: 0.3714 - accuracy: 0.8435
Epoch 6/25
1329/1329 [==============================] - 1s 829us/step - loss: 0.3704 - accuracy: 0.8437
Epoch 7/25
1329/1329 [==============================] - 1s 876us/step - loss: 0.3681 - accuracy: 0.8440
Epoch 8/25
1329/1329 [==============================] - 1s 979us/step - loss: 0.3672 - accuracy: 0.8437
Epoch 9/25
1329/1329 [==============================] - 1s 849us/step - loss: 0.3653 - accuracy: 0.8431
Epoch 10/25
1329/1329 [==============================] - 1s 832us/step - loss: 0.